#(실습-1) Elasticsearch 설치 및 실행

##실습 개요
1) 실습 목적 <br>
  이번 실습에서는 가장 많이 활용되고 있는 검색엔진 솔루션 중 하나인 Elasticsearch를 설치하고 구동해 봅니다. <br>
  데몬으로 구동한 후 가장 간단한 형태의 색인 및 검색 테스트를 통해 정상적으로 동작하는 지 확인합니다. <br>
2) 수강 목표
  * Elasticsearch를 노트북 환경에 설치할 수 있다.
  * Elasticsearch를 구동할 수 있다.
  * 간단한 색인 및 검색 명령을 수행할 수 있다.

### 실습 목차
* 1. Elasticsearch 설치
* 2. Elasticsearch 구동
* 3. 색인 및 검색 명령 실행

### 데이터셋 개요
* 데이터셋: cnn_dailymail
* 데이터셋 개요 : 약 27만개 cnn 뉴스 데이터

## 1. Elasticsearch 설치

In [1]:
# Elasticsearch Python 패키지 설치
!pip install elasticsearch #==8.8.0

In [ ]:
홈페이지 설치 링크
https://www.elastic.co/guide/en/elasticsearch/reference/current/targz.html

In [1]:
# Elasticsearch 8.8.0 다운로드 및 압축 풀기

# 리눅스용 엘라스틱서치 서버 설치를 위한 패키지 다운로드
#!wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.8.0-linux-x86_64.tar.gz
#!tar -xzf elasticsearch-8.8.0-linux-x86_64.tar.gz
#!ls elasticsearch-8.8.0/

###
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.15.1-linux-x86_64.tar.gz
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.15.1-linux-x86_64.tar.gz.sha512
!shasum -a 512 -c elasticsearch-8.15.1-linux-x86_64.tar.gz.sha512 
!tar -xzf elasticsearch-8.15.1-linux-x86_64.tar.gz
!cd elasticsearch-8.15.1/ 

--2024-09-20 09:55:26--  https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.15.1-linux-x86_64.tar.gz
Resolving artifacts.elastic.co (artifacts.elastic.co)... 34.120.127.130, 2600:1901:0:1d7::
Connecting to artifacts.elastic.co (artifacts.elastic.co)|34.120.127.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 605974468 (578M) [application/x-gzip]
Saving to: ‘elasticsearch-8.15.1-linux-x86_64.tar.gz’

elasticsearch-8.15. 100%[===================>] 577.90M  99.9MB/s    in 5.8s    

2024-09-20 09:55:32 (99.8 MB/s) - ‘elasticsearch-8.15.1-linux-x86_64.tar.gz’ saved [605974468/605974468]

--2024-09-20 09:55:32--  https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.15.1-linux-x86_64.tar.gz.sha512
Resolving artifacts.elastic.co (artifacts.elastic.co)... 34.120.127.130, 2600:1901:0:1d7::
Connecting to artifacts.elastic.co (artifacts.elastic.co)|34.120.127.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length:

In [2]:
# 코랩 노트북 환경에서 서버 구동을 위해서 PPID 1의 백그라운드 데몬 프로세스가 해당 폴더에 접근이 가능하도록 소유자 변경
#코랩용 !sudo chown -R daemon:daemon elasticsearch-8.8.0/

!useradd elasticsearch
!passwd elasticsearch
!chown -R elasticsearch:elasticsearch /home/code/elasticsearch-8.15.1/
!su - elasticsearch
!chown -R daemon:daemon /home/code/elasticsearch-8.15.1/

# 코랩 노트북 환경에서 서버 구동을 위한 리소스 제한/격리를 위해 아래 명령 수행
#!umount /sys/fs/cgroup
#!apt install cgroup-tools

New password: 
Retype new password: su: warning: cannot change directory to /home/elasticsearch: No such file or directory
$ 

## 2. Elasticsearch 구동

In [12]:
### 추가 코드
!/home/code/elasticsearch-8.15.1/bin/elasticsearch


[2024-09-20T09:15:34,129][ERROR][o.e.b.Elasticsearch      ] [instance-10464] fatal exception while booting Elasticsearchjava.lang.RuntimeException: can not run elasticsearch as root
	at org.elasticsearch.server@8.8.0/org.elasticsearch.bootstrap.Elasticsearch.initializeNatives(Elasticsearch.java:262)
	at org.elasticsearch.server@8.8.0/org.elasticsearch.bootstrap.Elasticsearch.initPhase2(Elasticsearch.java:161)
	at org.elasticsearch.server@8.8.0/org.elasticsearch.bootstrap.Elasticsearch.main(Elasticsearch.java:66)

See logs for more details.

ERROR: Elasticsearch did not exit normally - check the logs at /home/code/elasticsearch-8.8.0/logs/elasticsearch.log

ERROR: Elasticsearch exited unexpectedly


In [15]:
# 엘라스틱서치의 데몬 인스턴스 만들기
import pwd
import os
from elasticsearch import Elasticsearch, helpers
import numpy as np
import pandas as pd
import json
from subprocess import Popen, PIPE, STDOUT

# 사용자 ID를 'elasticsearch'로 변경하는 함수
def change_user_to_elasticsearch():
    user_info = pwd.getpwnam('elasticsearch')  # 'elasticsearch' 사용자 정보 가져오기
    os.setuid(user_info.pw_uid)  # 사용자 ID 변경

# Elasticsearch 프로세스를 실행
es_server = Popen(['/home/code/elasticsearch-8.15.1/bin/elasticsearch'],
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=change_user_to_elasticsearch  # 프로세스 실행 전에 사용자 ID 변경
                 )

# 인스턴스를 로드하는 데 약간의 시간이 걸림
import time
time.sleep(30)

In [17]:
# 데몬이 구동되었는지 확인 (세개의 daemon process가 있어야 함)
!ps -ef | grep elasticsearch

root      639451  639333  0 20:22 pts/5    00:00:00 su - elasticsearch
root      640203  639836  0 20:24 pts/6    00:00:00 su - elasticsearch
elastic+  646208  639452 19 20:53 pts/5    00:00:03 /home/code/elasticsearch-8.15.1/jdk/bin/java -Xms4m -Xmx64m -XX:+UseSerialGC -Dcli.name=server -Dcli.script=./elasticsearch -Dcli.libs=lib/tools/server-cli -Des.path.home=/home/code/elasticsearch-8.15.1 -Des.path.conf=/home/code/elasticsearch-8.15.1/config -Des.distribution.type=tar -cp /home/code/elasticsearch-8.15.1/lib/*:/home/code/elasticsearch-8.15.1/lib/cli-launcher/* org.elasticsearch.launcher.CliToolLauncher
elastic+  646279  646208 99 20:53 pts/5    00:00:57 /home/code/elasticsearch-8.15.1/jdk/bin/java -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -Djava.security.manager=allow -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true 

In [18]:
# 데몬 구동후 password 설정 단계 필요
# 명령 실행 후 "Please confirm that you would like to continue"에서 y 입력 필요
# 코랩용 !/content/elasticsearch-8.8.0/bin/elasticsearch-setup-passwords auto -url "https://localhost:9200"
# ai stages용
!/home/code/elasticsearch-8.15.1/bin/elasticsearch-setup-passwords auto -url "https://localhost:9200"


Failed to authenticate user 'elastic' against https://localhost:9200/_security/_authenticate?pretty
Possible causes include:
 * The password for the 'elastic' user has already been changed on this cluster
 * Your elasticsearch node is running against a different keystore
   This tool used the keystore at /home/code/elasticsearch-8.15.1/config/elasticsearch.keystore

You can use the `elasticsearch-reset-password` CLI tool to reset the password of the 'elastic' user


ERROR: Failed to verify bootstrap password, with exit code 78


In [19]:
username = 'elastic'

# 위 명령 실행 결과의 마지막 부분인 PASSWORD elastic 값으로 교체 필요
password = ''

es = Elasticsearch(['https://localhost:9200'], basic_auth=(username, password), ca_certs="/home/code/elasticsearch-8.15.1/config/certs/http_ca.crt")

resp = dict(es.info())

resp

{'name': 'instance-10464',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'rX591VGTQEKc8qjGlWCxXQ',
 'version': {'number': '8.15.1',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': '253e8544a65ad44581194068936f2a5d57c2c051',
  'build_date': '2024-09-02T22:04:47.310170297Z',
  'build_snapshot': False,
  'lucene_version': '9.11.1',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

## 3. 색인 및 검색 명령 실행

In [11]:
# 데이터셋 사용을 위한 huggingface datasets 패키지 인스톨
!pip install datasets -q

In [12]:
# 실제 다운로드 및 데이터셋 로딩
import datasets
dataset = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train").to_pandas()
dataset.drop("id", axis=1, inplace=True)
print(f"shape of dataset: {dataset.shape}")
dataset.head()

/home/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 11490/11490 [00:00<00:00, 77091.43 examples/s]


shape of dataset: (287113, 2)


,article,highlights
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa..."
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non..."
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical..."


In [13]:
# Elasticsearch 인덱스의 설정 및 매핑 정의
settings = {
    "settings":{
        "number_of_shards":1,
        "number_of_replicas":0
    },
    "mappings":{
        "properties":{
            "article":{
                "type":"text"
            },
            "highlights":{
                "type":"text"
            }
        }
    }
}

In [14]:
def json_formatter(dataset, index_name):
    """
    이 함수는 Elasticsearch 색인을 위한 JSON 형식의 딕셔너리를 생성하는 데 사용됩니다.

    Args:
      dataset: 이 함수를 적용하려는 데이터입니다.
      index_name: Elasticsearch의 인덱스 이름입니다.
    """
    try:
        List = []
        columns = dataset.columns
        for idx, row in dataset.iterrows():
            dic = {}
            dic['_index'] = index_name
            source = {}
            for i in dataset.columns:
                source[i] = row[i]
            dic['_source'] = source
            List.append(dic)
        return List

    except Exception as e:
        print("There is a problem: {}".format(e))

In [15]:
MY_INDEX = es.indices.create(index="news_index", body=settings)
MY_INDEX

/tmp/ipykernel_624645/1490638917.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  MY_INDEX = es.indices.create(index="news_index", body=settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'news_index'})

In [16]:
# 너무 많아서 100개만 사용
dataset = dataset[:100]

json_Formatted_dataset = json_formatter(dataset=dataset, index_name='news_index')
json_Formatted_dataset[0]

{'_index': 'news_index',
 '_source': {'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his numbe

In [17]:
# 색인을 위해 elasticsearch.helpers API를 사용합니다.
res = helpers.bulk(es, json_Formatted_dataset[:100])
res

(100, [])

In [18]:
# 색인된 문서중 10개의 데이터 샘플 가져오기
query = es.search(
    index="news_index",
    body={
      "size":10,
      "query": {
        # 모든 문서가 매치된다는 의미 (_socre는 모두 1.0)
        "match_all":{}
      }
    }
)

output = pd.json_normalize((query['hits']['hits']))
output

/tmp/ipykernel_624645/2123980155.py:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  query = es.search(


,_index,_id,_score,_source.article,_source.highlights
0,news_index,GMDqDpIBhyBJtTYKrreR,1.0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...
1,news_index,GcDqDpIBhyBJtTYKrreR,1.0,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...
2,news_index,GsDqDpIBhyBJtTYKrreR,1.0,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa..."
3,news_index,G8DqDpIBhyBJtTYKrreR,1.0,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non..."
4,news_index,HMDqDpIBhyBJtTYKrreR,1.0,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical..."
5,news_index,HcDqDpIBhyBJtTYKrreR,1.0,"BAGHDAD, Iraq (CNN) -- Dressed in a Superman s...","Parents beam with pride, can't stop from smili..."
6,news_index,HsDqDpIBhyBJtTYKrreR,1.0,"BAGHDAD, Iraq (CNN) -- The women are too afrai...","Aid workers: Violence, increased cost of livin..."
7,news_index,H8DqDpIBhyBJtTYKrreR,1.0,"BOGOTA, Colombia (CNN) -- A key rebel commande...",Tomas Medina Caracas was a fugitive from a U.S...
8,news_index,IMDqDpIBhyBJtTYKrreR,1.0,WASHINGTON (CNN) -- White House press secretar...,"President Bush says Tony Snow ""will battle can..."
9,news_index,IcDqDpIBhyBJtTYKrreR,1.0,(CNN) -- Police and FBI agents are investigati...,Empty anti-tank weapon turns up in front of Ne...


#Reference

## Required Package

elasticsearch==8.8.0 <br>
datasets  <br>




## 콘텐츠 라이선스

저작권 : <font color='blue'> <b> ©2023 by Upstage X fastcampus Co., Ltd. All rights reserved.</font></b>

<font color='red'><b>WARNING</font> : 본 교육 콘텐츠의 지식재산권은 업스테이지 및 패스트캠퍼스에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다. </b>